### We are assuming that the state vector are in the following order:
$$ x = [E_{qp}\quad E_{dp}\quad \delta\quad \omega\quad V_F\quad V_A\quad V_E]^T $$

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from rushisland2 import *

In [3]:
who

Aex1	 Aex2	 Bex1	 Bex2	 H	 KA1	 KA2	 KC1	 KC2	 
KD	 KD1	 KD2	 KE1	 KE2	 KF1	 KF2	 Pm1	 Pm2	 
Rs	 S2_mat	 S3_mat	 T2_mat	 T3_mat	 TA1	 TA2	 TE1	 TE2	 
TF1	 TF2	 Tdp	 Tqp	 Trissian	 Xd	 Xdp	 Xl	 Xq	 
Xqp	 f_Edp1	 f_Edp2	 f_Efd	 f_Eqp1	 f_Eqp2	 f_VA1	 f_VA2	 f_VE1	 
f_VE2	 f_VF1	 f_VF2	 f_Vdq	 f_delta1	 f_delta2	 f_w1	 f_w2	 nd	 
np	 ratio	 sys_fun	 


In [4]:
import numdifftools as nd
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy import optimize
np.set_printoptions(precision=25)

In [5]:
x = np.array([9.8702904764475974e-01,6.0720200986568440e-01,1.2488635337278593e+00,
 3.9513198430343923e-29, 4.8356743414247063e+00, 5.3729714904718956e-03,
 2.3129894806041311e+00, 9.0826094936352175e-01, 6.4022044420710678e-01,
 1.3157812680797303e+00, 3.7013504926515539e-21, 2.5467521593523479e+00,
 4.7162077025043478e-02, 2.1398641868916126e+00])
# x = np.append(x,[5.769953284494019,5.400027691631498,1.045418310397626,1.077334874177618])

sol = optimize.root(sys_fun, x, method='hybr')
print(sol.fun)
print(sol.message)
print(sol.x)
print(sol.success)
x = sol.x

[-8.3318801097572719e-17 -3.1720657846433040e-16 -3.0997013965550357e-28
  4.9508273115949005e-18  0.0000000000000000e+00  2.9683046144492032e-15
  0.0000000000000000e+00  0.0000000000000000e+00 -4.2294210461910723e-16
  1.2545912538345359e-26  1.4852481934784702e-17 -3.7318420995803582e-16
  6.0338207860062865e-17  4.4408920985006262e-16]
The solution converged.
[ 9.8702904764637900e-01  6.0720200986339867e-01  1.2488635337247085e+00
 -8.2222133228430873e-31  4.8356743414294767e+00  5.3729714904771960e-03
  2.3129894806063538e+00  9.0826094936361890e-01  6.4022044420757318e-01
  1.3157812680813068e+00  3.3279066601270862e-29  2.5467521593576761e+00
  4.7162077025142149e-02  2.1398641868961223e+00]
True


In [6]:
np.abs(f_Vdq(x))

array([[1.04001239086476  ],
       [1.0311200125534163]])

# Notations

$$ \dot{z} = f(z),\quad f(z_0)=0 $$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  \Delta z^T H \Delta z + H.O.T(z)$$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  Z_2*B_z^2 + H.O.T(z)$$
Let $w = Q\Delta z$, and ignore the $\Delta$ sign then
$$Q^{-1}\dot{w} = JQ^{-1} w + \frac{1}{2!} (Q^{-1}w)^T H  (Q^{-1}w) + H.O.T(Q^{-1}w)$$
$$\dot{w} = QJQ^{-1} w + Q \frac{1}{2!} w^T (Q^{-T}H Q^{-1})w + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!} w^T H_Q w + Q * H.O.T(Q^{-1}w)$$
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!}  \begin{bmatrix} w^T H_Q[0] w \\  w^T H_Q[1] w \\ ...  \end{bmatrix} + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
where $W_2$ $W_3$ are constant matrices and $B_w^2$ and $B_w^3$ are column vectors deriving from $w \otimes w$ and $w \otimes w \otimes w$

And we assume that $$w = [u \quad v]^T$$ where $u$ is center variable, $v$ is stable variable

Now assume $B^2_{u}$ is $u\otimes u$ (note that this is a long column vector, so we do not need the $vec$ sign) but ij terms only appear once: e.g. if $u = [u_1 u_2 u_3]^T$, then $B^2_{u}=[u_1^2, u_2u_1, u_3u_1, u_2^2, u_3u_2, u_3^2]$

So we also need two transformation matrices $T$ and $S$ which doing transformations between $B_{uu}$ and $u\otimes u$ like: $$B^2_{u}=T_2 (u\otimes u)$$  
$$u\otimes u=S_2 B^2_{u}$$ 

similarly we will have  
$$B^3_{u}=T_3 (u\otimes B^2_{u})$$  
$$(u\otimes B^2_{u})=S_3 B^3_{u}$$ 

### Now we calculate Jacobian matrix $J$ and  the tranformation matrix $Q$

In [7]:
n_c = 4
n_s = 10
n = x.shape[0]

# Compute Jacobian Matrix J and its eigenvalues
all_fun = [f_Eqp1,f_Edp1,f_delta1,f_w1,f_VF1,f_VA1,f_VE1,f_Eqp2,f_Edp2,f_delta2,f_w2,f_VF2,f_VA2,f_VE2]
J = np.array([nd.Jacobian(f)(x).ravel() for f in all_fun])
#display(pd.DataFrame(J))
lam, v = np.linalg.eig(J.T) # Here we want to calculate the left eigenvecs, so use J_org.T; because use left eigvec makes it easier to calc transformation matrix
print(lam)

[-1.0099204150831989e+01+15.526684726645993j
 -1.0099204150831989e+01-15.526684726645993j
 -7.1529032193935471e-01 +5.7018807049158795j
 -7.1529032193935471e-01 -5.7018807049158795j
 -7.4993344867380074e-12 +3.843749319338007j
 -7.4993344867380074e-12 -3.843749319338007j
 -5.0397362357964761e+00 +0.j
 -3.5682221066757336e-13 +1.2915153162562112j
 -3.5682221066757336e-13 -1.2915153162562112j
 -3.2400622535479413e+00 +0.j
 -1.4224176437018947e+00 +0.8106136960057834j
 -1.4224176437018947e+00 -0.8106136960057834j
 -1.9715384390348540e+00 +0.j
 -8.3619426557698695e-01 +0.j                ]


In [9]:
Q = np.block([
    [np.stack((v[:,4].real,v[:,4].imag,v[:,7].real,v[:,7].imag))],
    [v[:,[0,1,2,3,6,9,10,11,12,13]].T]
])
J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
# display(pd.DataFrame(J_cs))

In [ ]:
# # Compute transformation matrix Q
# Q1 = np.block([
#     [np.stack((v[:,4].real,v[:,4].imag,v[:,7].real,v[:,7].imag))],  # Here we know the 4-th and 8-th eigenvalues are what we interested
#     [np.eye(10),np.zeros((10,4))]
# ])
# J1 = np.dot(np.dot(Q1,J),np.linalg.inv(Q1))
# # display(pd.DataFrame(J1))
# Q2 = np.block([
#     [np.eye(4), np.zeros((4,10))],
#     [sp.linalg.solve_sylvester(J1[4:,4:], -J1[0:4,0:4], J1[4:,0:4]), np.eye(10)] #AX+XB=Q
# ]) 
# Q = np.dot(Q2,Q1)
# #J_cs = np.dot(np.dot(Q,J_org),np.linalg.inv(Q))
# J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
# del Q1
# del Q2
# del J1
# display(pd.DataFrame(J_cs))

### Now we can 2nd and 3rd order matrices $W_2$ and $W_3$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
Since originally we are calculating hessian matrices of $f(z)$, then we need to make transformation from $z$ to $w$, thus
$$ \frac{1}{2} Q Z_2 B_z^2 = \frac{1}{2} Q Z_2[T_2 (z \otimes z)] = \frac{1}{2} Q Z_2 T_2 (Q^{-1} w \otimes Q^{-1} w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})(w\otimes w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 B_w^2 = W_2 B_w^2$$
Thus
$$\frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 = W_2 $$

For 3rd order
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (z \otimes B_z^2) = \frac{1}{3!} Q Z_3 T_3 (z \otimes T_2(z \otimes z))$$
where

\begin{align}
&  z \otimes T_2(z \otimes z) \\
& = (I\otimes T_2)(z \otimes z \otimes z) \\
& = (I\otimes T_2)(Q^{-1}w \otimes Q^{-1}w \otimes Q^{-1}w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes w \otimes w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes S_2 B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 )(w \otimes B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3
\end{align}

Thus
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3 = W_3 B_w^3$$


In [10]:
Z2 = np.zeros((n,n*(n+1)//2))
for i in range(n):
    hes = nd.Hessian(all_fun[i])(x) # The original Hessian of each f in all_fun
    hes[np.triu_indices(n,1)] *= 2.0  # double each element above the main diagonal
    Z2[i] = hes[np.triu_indices(n)] # Keep upper triangular part
Z2 = Z2/2.0 #divide all elements by 2, which corresponds to *2 in above line
# display(pd.DataFrame(Z2))

In [11]:
W2 = np.linalg.multi_dot([Q, Z2, T2_mat(n), np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), S2_mat(n)])
display(pd.DataFrame(W2)) # W2 is of dim: n x n(n+1)/2 = 14 x 105

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
0,(3.831931749232753+5.160736907163239e-15j),(17.02120829766051+1.1999019998175995e-14j),(-4.146849955764004+6.941061998721765e-16j),(-3.6974356081549504-1.6931760185802543e-15j),(0.0434754723630732+0.21978339942362765j),(0.04347547236306473-0.21978339942364128j),(-3.0694256351347557+14.623186635284044j),(-3.069425635134754-14.623186635284053j),(-7.755375687174622-5.732683957803035e-15j),(4.017618896822957-5.476580328794257e-15j),(-2.200878834921979-4.8862207839227265j),(-2.200878834921971+4.8862207839227185j),(1.4840168833232557+9.354885142639366e-16j),(0.09406940757882704-4.667148488520324e-18j),(63.52833477449454+5.465605501661776e-15j),(9.913499059012505+4.216118277000902e-15j),(0.6559639984400618-8.116447924713856e-16j),(0.30362261244800653+1.3077598415878413j),(0.3036226124480056-1.307759841587867j),(-0.019842730650765073-6.8927629323541995j),(-0.019842730650784283+6.892762932354211j),(-22.188260101219907-8.49506597335176e-15j),(-18.493020606375502-8.347771953910058e-15j),(-1.6499449997930757-8.207881531615035j),(-1.6499449997930629+8.207881531615012j),(5.903912314712445+9.458459777504944e-16j),(-0.14373029122539013-5.079202951813792e-17j),(1.367856474919734+1.9486658075681774e-16j),(1.841598723321566+2.287431392584759e-16j),(0.06180217900786888-0.017409750007215546j),(0.06180217900787218+0.017409750007218328j),(1.0561734842621326-6.782810585570608j),(1.0561734842621302+6.782810585570609j),(1.8769949930086307-9.713840114083004e-16j),(-2.3077882016537483+1.7633865272671832e-16j),(0.8147770986906411+1.4773115348106765j),(0.8147770986906377-1.477311534810677j),(0.7605561281174044-4.6028841921883573e-17j),(-0.05771844007189573-4.8314918090394054e-18j),(0.6762503937809595+3.066290478317617e-16j),(-0.0013166823825886765-0.12455043093569526j),(-0.0013166823825859883+0.12455043093569856j),(1.369659871441596-4.796754698441616j),(1.3696598714415933+4.796754698441618j),(3.490184007998411+1.0197530004291886e-15j),(-1.915921824359658+9.44518128882055e-16j),(0.5699953930508544+1.6689858777361222j),(0.5699953930508531-1.66898587773612j),(-0.27881835620371265-1.5529689355437506e-16j),(-0.036053147190022033-8.797881060217384e-18j),(0.022397792273980848+0.054277493783922506j),(0.11957940709527974-1.4497025600508938e-16j),(-0.432053745151738-0.04358368548098969j),(0.4298214323335745+0.11379279996047628j),(-0.3010497148762853-0.28958444154550034j),(-0.07358651864942367-0.16126348892015185j),(0.11518671554703461-0.1754294319443054j),(-0.23120293267693864-0.16881178835374958j),(0.016566605199478175+0.042363528733399304j),(-5.838806195097749e-05+0.002895497961800414j),(0.022397792273980425-0.054277493783922284j),(0.42982143233358727-0.11379279996048203j),(-0.4320537451517472+0.04358368548100279j),(-0.3010497148762792+0.28958444154551355j),(-0.07358651864942581+0.16126348892015097j),(-0.23120293267694372+0.1688117883537551j),(0.11518671554704121+0.17542943194430313j),(0.016566605199478817-0.04236352873340058j),(-5.838806195105018e-05-0.002895497961800476j),(-8.201466081422224-5.096718454715357j),(14.81555304811953+2.221028013482396e-15j),(-0.5670003259202611-12.701904474538651j),(1.4766730945816282+7.051870191750272j),(6.897288672370146-0.5051426883824499j),(-4.442307639186418-2.6559621322992286j),(1.3122943695503693-0.0447850199851995j),(-0.04119669318651912+0.1295332420015767j),(-8.201466081422229+5.096718454715356j),(-0.5670003259202608+12.701904474538651j),(1.4766730945816255-7.0518701917502735j),(-4.442307639186422+2.6559621322992344j),(6.897288672370143+0.5051426883824446j),(1.31229436955037+0.0447850199852009j),(-0.041196693186519064-0.12953324200157681j),(2.156717950892765+2.3145580058388845e-16j),(3.0247849270923894+4.971566794902

In [10]:
# Z3 = np.zeros((n,(n*(n+1)*(n+2)//6)))
# Z3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
# for i in range(n):
#     t = Trissian(all_fun[i], x)
#     Z3[i] = [t[j] for j in Z3_idx]

# Z3_Gain = []
# for i in Z3_idx:
#     val = 1
#     for j in range(n_c):
#         val *= math.factorial(i.count(j))
#     Z3_Gain.append(val)
# Z3_Gain = np.diag(1/np.array(Z3_Gain))    
# Z3 = Z3.dot(Z3_Gain)
# np.savetxt("Fxxx_goodcase2.csv", Z3, delimiter=",")

In [12]:
Z3 = np.genfromtxt('Fxxx_goodcase2.csv', delimiter=',',dtype=float)
# display(pd.DataFrame(Z3))

In [13]:
W3 = np.linalg.multi_dot([Q, Z3, T3_mat(n), np.kron(np.eye(n),T2_mat(n)), 
                      np.kron(np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), np.linalg.inv(Q)), 
                      np.kron(np.eye(n),S2_mat(n)), S3_mat(n)])

### Now we are able to calculate center manifold up to 2nd order terms, which we denote it by:
$$v = h_{c2}(u) = \color{red}{H_{c2}}B_u^2$$
where $\color{red}{H_{c2}} \in \mathbb{R}^{n_s \times n_c(n_c+1)/2}$ is the matrix we want to find; the subscript $c$ means center manifold (to distinguish it from normal form later), and 2 means 2nd order

Then according to equation:
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
if we express the term $W_2 B_w^2$ in form of $u$ and $v$, we will have
$$ \dot{u} = J_c u + f_2(u,v) = J_c u + \color{red}{U_u^2} B_u^2 + \color{red}{U_{uv}} B_{uv} + U_v^2 B_v^2 + ... $$
$$ \dot{v} = J_s v + g_2(u,v) = J_s v + \color{red}{V_u^2} B_u^2 + V_{uv} B_{uv} + V_v^2 B_v^2 + ... $$

Since we can find $\color{red}{H_{c2}}$ by solving the sylvester equation: $\color{red}{J_s H_{c2} + H_{c2} C_2 = -V_u^2}$ so we need to calculate the matrix of $V_u^2$

where $C_2$ comes from
$$ (DB_u^2)J_c u = T_2 D(u \otimes u) J_c u = T_2 \bar{J_c}(u \otimes u) = T_2 \bar{J_c} S_2 B_u^2 = -C_2 B_u^2$$
and
$$\bar{J_c} = (I_n \otimes J_c) + \begin{pmatrix} I_n \otimes J_1\\I_n \otimes J_2 \\ ... \\I_n \otimes J_n \end{pmatrix}$$ where each $J_i$ is the i-th row of matrix $J_c$

Now we construct matrix <font color = 'red'>$V_u^2$, $\bar{J_c}$ $C_2$ $H_{c2}$ </font>

In [14]:
J_c = J_cs[0:n_c,0:n_c]
J_s = J_cs[n_c:,n_c:]
V2_uu = np.zeros((n_s,n_c*(n_c+1)//2))


W2_idx = [(i,j) for i in range(n) for j in range(i,n)]
V2_uu_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
V2_uu_idx = [W2_idx.index(i) for i in V2_uu_idx]
V2_uu = np.array([w[V2_uu_idx] for w in W2[n_c:n]])
# display(pd.DataFrame(V2_uu))

J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
#display(pd.DataFrame(J_cbar))
C2 = - np.linalg.multi_dot([T2_mat(n_c),J_cbar,S2_mat(n_c)])
#display(pd.DataFrame(C2))

H_c2 = sp.linalg.solve_sylvester(J_s, C2, -V2_uu)
# display(pd.DataFrame(H_c2))

### Now we can plug $v = {H_{c2}}B_u^2$ into equation of $\dot{u}$, then we can get up to the 3rd order terms
Since we have 
$$ \dot{u} = J_c u + f_2(u,v) + f_3(u,v) + ... $$
by plugging 
$$v = h_{c2}(u) = H_{c2} B_u^2$$
into the above equation, for third order terms of $u$, we only need:
$$ \dot{u} = J_c u + f_2(u,H_{c2} B_u^2) + f_3(u,0) + ... $$
Let
$$f_2(u,v) = U_u^{2} B_u^2 + U_{uv} B_{uv} + U_v^{2} B_v^2$$
and 
$$f_3(u,0) = U_u^{3} B_u^3$$
Then we can see that $B_{uv}$ is in 3rd order of $u$. $B_v^2$ is in 4th order of $u$, which is ignored here.
So the center manifold up to 3rd order of $u$ could be expressed as:
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} B_{uv} + U_u^{3} B_u^3 + O(u^4) $$
Since 
$$B_{uv} = u \otimes v = u \otimes H_{c2} B_u^2 = (I_{n_c} \otimes H_{c2})(u \otimes B_u^2) = (I_{n_c} \otimes H_{c2})S_3 B_u^3$$
Thus
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} (I_{n_c} \otimes H_{c2})S_3 B_u^3 + U_u^{3} B_u^3 + O(u^4) = J_c u + \color{red}{U_u^{2}} B_u^2 + [\color{red}{U_{uv}} (I_{n_c} \otimes H_{c2})\color{red}{S_3} + \color{red}{U_u^{3}}] B_u^3 + O(u^4)$$

So here in order to get the know the equation of $\dot{u}$ up to 3rd order, we need to know the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$, $U_u^{3}$</font>.

Here in order to know $U_u^{3}$, which comes from $f_3(u,0)$, we need to calculate matrix $W_3$ in equation $\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$

### Firstly we calculate $U_u^2$ and $U_{uv}$. Both of them comes from $W_2$

In [15]:
W2_idx = [(i,j) for i in range(n) for j in range(i,n)]
Uu2_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
Uu2_idx = [W2_idx.index(i) for i in Uu2_idx]
Uu2 = np.array([w[Uu2_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uu2))
Uuv_idx = [(i,j) for i in range(n_c) for j in range(n_c,n)]
Uuv_idx = [W2_idx.index(i) for i in Uuv_idx]
Uuv = np.array([w[Uuv_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uuv))

### Now we are ready to calculate matrix $U_u^3$. 
But in order to calculate it, we need to calculate the 3rd order derivative matrix $W_3$, then formulize $U_u^3$

Now we have known the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$  and $U_u^{3}$ </font>

Then we can calculate 
$$U_{uc}^3 = U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}$$ 
in the center manifold:
$$\dot{u} =  J_c u + U_u^{2} B_u^2 + [U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}] B_u^3 + O(u^4) = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$

In [16]:
W3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
Uu3_idx = [(i,j,k) for i in range(n_c) for j in range(i,n_c) for k in range(j,n_c)]
Uu3_idx = [W3_idx.index(i) for i in Uu3_idx]
Uu3 = np.array([w[Uu3_idx] for w in W3[0:n_c]])
#display(pd.DataFrame(Uu3))
Uuc3 = np.linalg.multi_dot([Uuv, np.kron(np.eye(n_c), H_c2), S3_mat(n_c)]) + Uu3

### <font color='red'>Now we've got the flow $\dot{u}$ on the center manifold up to 3rd order, the next step is to calculate the normal form</font>

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
至此，我们只需要研究这个方程，我们已经知道$J_c$, $U_u^{2}$, $U_{uc}$,那么接下来就是要算normalform
</font>
#### First, we calculate the normal form up to the 2nd order:
$$u = y + \color{red}{H_{2n}} B_y^2$$
where we want to find matrix $\color{red}{H_{2n}}\in \mathbb{R}^{n_c \times n_c(n_c+1)/2}$

Then by plugging it into the above equation, we will get:
$$(I + H_{2n} DB_y^2) \dot{y} = J_c (y + H_{2n} B_y^2) + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
<font color='red'>$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since by some derivation, we know:
$$B_u^2 = B_y^2 + T_2\bar{H}(I\otimes S_2)S_3 B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ ($T_2=T_2(n_c)$) and $h_1$ ... $h_n$ are rows of matrix $H$.
In the above equations, if we assume $u\in \mathbb{R}^{n_c}$, then $H_{2n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)}{2}}$, $H \in \mathbb{R}^{n_c\times n_c^2}$, $\bar{H} \in \mathbb{R}^{n_c^2 \times n_c^3}$, $S_2=S_2(n_c)$, $S_3=S_3(n_c)$

Then
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
**Then up to 2nd order terms on the right hand side is**:
$$\dot{y} = J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 - H_{2n} DB_y^2 J_c y + O(y^3)$$
If we denote $- DB_y^2 J_c y = \color{red}{C_2} B_y^2$ as we did in calculation of center manifold, then:
$$\dot{y} = J_c y + [J_c H_{2n} + U_u^2 + H_{2n} C_2] B_y^2 + O(y^3)$$
Due to resonance terms, $J_c H_{2n} + H_{2n} C_2 + U_u^2 $ might not be totally eliminated, so we let
<font color='red'>$$J_c H_{2n} + H_{2n} C_2 = - U_u^2 + R_{2n} $$</font>
where matrix $R_{2n}$ represents resonance terms in normal form.

Now the problem is trying to solve for $H_{2n}$ which makes $R_{2n}$ as simple as possible, where $J_c$, $U_{u}^2$ and $C_2$ are known

Since $J_c \in \mathbb{R}^{n_c\times n_c}$ , $H_{2n} \in \mathbb{R}^{n_c\times \frac{n_c(n_c+1)}{2}}$  and $C_2 \in \mathbb{R}^{\frac{n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)}{2}}$
First we could transform the equation into:
$$\underbrace{(I_{\frac{n_c(n_c+1)}{2}}\otimes J_c + C_2^T \otimes I_{n_c})}_\text{$\Lambda_2$} \cdot \text{vec} (H_{2n}) = -\text{vec}(U_u^2) + \text{vec}(R_{2n})$$


Since we know that for Hopf Bifurcation, the 2nd order terms in normal form will all be eliminated, so $\Lambda$ should be a full rank matrix and the resonance term $R_{2n}$ will not exist, which is also verified below. Then we could solve for $H_{2n}$ by directly solving the Sylvester equation $J_c H_{2n} + H_{2n} C_2 = - U_u^2$

In [17]:
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2

In [18]:
# dummy1, dummy2 = np.linalg.eig(Lambda2)
# print(dummy1)
display(pd.DataFrame(R_2n))

,0,1,2,3,4,5,6,7,8,9
0,(1.5987211554602254e-14-5.566952357873342e-14j),(-4.618527782440651e-14+1.4801480363952883e-14j),(-1.9539925233402755e-14+2.2281348749211033e-17j),(4.440892098500626e-16+7.258537738534562e-15j),(-2.842170943040401e-14+1.1855045487466116e-14j),(1.7763568394002505e-15+1.3887238488101945e-15j),(7.771561172376096e-15+7.555188385144736e-15j),(-6.661338147750939e-16-8.424682661535006e-16j),(8.881784197001252e-16+3.7532554005311355e-17j),(-1.1102230246251565e-16-5.167349463044228e-16j)
1,(4.7503667666148885e-14+2.9596311878380616e-14j),(-3.544387006115812e-14+1.528377950665933e-14j),(-2.6201263381153694e-14+9.106448083109805e-15j),(-2.4424906541753444e-15-1.0616723805727816e-14j),(1.4210854715202004e-14-5.1152312264610954e-14j),(3.9968028886505635e-15+6.2939246887445e-15j),(-7.105427357601002e-15+2.0925247994242107e-15j),(1.1102230246251565e-16-2.7835624144271417e-16j),3.7649528692498964e-16j,(-8.881784197001252e-16+1.5398350764702575e-16j)
2,(-1.9095836023552692e-13-1.8589768024262947e-13j),(-2.2026824808563106e-13-9.399920714814568e-14j),(1.4210854715202004e-13-7.380867598863802e-14j),(1.509903313490213e-13+7.594962819374287e-14j),(8.526512829121202e-14-7.936163012960611e-14j),(7.105427357601002e-15-2.7051818665074417e-14j),(1.0658141036401503e-14-7.338424384809824e-14j),(-8.881784197001252e-16+2.5578248633416327e-15j),(4.440892098500626e-15+9.96104588524358e-16j),(-1.3322676295501878e-15+2.0361046918723161e-16j)
3,(6.927791673660977e-14+2.6834306835248077e-14j),(-1.4210854715202004e-13+9.247459824725001e-14j),(-8.375244942016025e-14-2.091877262756148e-14j),(6.722400414105323e-14+2.1366599364542524e-15j),(8.526512829121202e-14-5.365241328258271e-14j),(2.842170943040401e-14+6.804689813536331e-15j),(-1.0658141036401503e-14-9.703892203535998e-15j),(-1.5543122344752192e-15+5.894124521467479e-16j),-2.2715962201639686e-15j,(2.6645352591003757e-15+3.5183067875882072e-15j)


### Next, we calculate the normal form up to the 3rd order terms
From previous equation:
$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since if we only consider 3rd order terms, $U_{uc}^3 B_u^3$ can be expressed as:
$$ U_{uc}^3 B_u^3 = U_{uc}^3 B_y^3$$
So 
$$\dot{y} = [I - \underbrace{H_{2n} DB_y^2}_\text{O(y)} + \underbrace{(H_{2n} DB_y^2)(H_{2n} DB_y^2)}_\text{O(y^2)}+O(y^3)] [J_c y + \underbrace{(J_c H_{2n} + U_u^2) B_y^2}_\text{O(y^2)} + \underbrace{[U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3}_\text{O(y^3)} + O(u^4)]$$
Thus the 3rd order terms on the right hand side is:

\begin{align}
& [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 (J_c H_{2n} + U_u^2) B_y^2 + (H_{2n} DB_y^2)(H_{2n} DB_y^2)J_c y \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 - H_{2n} DB_y^2 J_c y] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 + H_{2n} C_2 B_y^2] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} \underbrace{DB_y^2 R_{2n} B_y^2}_\text{O(y^3)} 
\end{align}

where the sizes of matrices are:
\begin{align}
& H_{2n}, U_{u2} --  n_c \times \frac{n_c(n_c+1)}{2}  \\
& T_2(n_c) -- \frac{n_c(n_c+1)}{2} \times n_c^2  \\
& \bar{H}, \bar{R} -- n_c^2 \times n_c^3  \\
& I\otimes S_2(n_c) -- (n_c \times n_c) \otimes (n_c^2 \times \frac{n_c(n_c+1)}{2})  \\
& S_3(n_c) -- \frac{n_c^2 n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
& U_{uc}^3 -- n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
\end{align}

So we only need to calculate 
$$ DB_y^2 R_{2n} B_y^2 = T_2 \bar{R} (I \otimes S_2) S_3 B_y^3$$
where $$\bar{R} = (I_n\otimes R) + \begin{pmatrix} I_n\otimes r_1 \\ I_n\otimes r_2 \\ \vdots \\ I_n\otimes r_n \end{pmatrix} $$
where $R = R_{2n}T_2$ and $r_i$ are rows of $R$

Thus the 3rd order terms are:
$$N_y^3 B_y^3 = [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3 - H_{2n} T_2 \bar{R} (I \otimes S_2) S_3 ]B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ and $h_1$ ... $h_n$ are rows of matrix $H$

Now the normal form is:
$$\dot{y} = J_c y + R_{2n}B_y^2 + N_y^3 B_y^3 + O(y^4)$$

Now we want to simply $N_y^3$ as many as possible, so we consider another 3rd order nonlinear transformation 
$$ y = z + H_{3n}B_z^3$$

Here if we assume $y,z \in \mathbb{R}^{n_c}$ then $H_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$

Similarly, we will have 
$$\dot{z} = J_c z + R_{2n}B_z^2 + R_{3n} B_z^3 + O(z^4)$$
where
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
where 
$$C_3 B_z^3 = -DB_z^3 J_c z$$ and $$C_3 = -T_3(I_n\otimes T_2)\bar{J}(I_n\otimes S_2)S_3$$
where
$$\bar{J} = (I_{n^2} \otimes J_c) + I_n \otimes \begin{pmatrix} I_n\otimes jc_1 \\I_n\otimes jc_2 \\ \vdots \\ I_n\otimes jc_n \\ \end{pmatrix} + \begin{pmatrix} I_{n^2}\otimes jc_1 \\I_{n^2}\otimes jc_2 \\ \vdots \\ I_{n^2}\otimes jc_n \\ \end{pmatrix}$$

$jc_i$ are rows of matrix $J_c$

So here we have $R_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, $C_3 \in \mathbb{R}^{\frac{n_c(n_c+1)(n_c+2)}{3!} \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, 

So in order to calculate $R_{3n}$, first we need to calculate $N_y^3$.
Let's calcuate $\bar{H}$ and $\bar{R}$ first

In [19]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uuc3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(Ny3))
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(C3))

Then we tranform the equation:
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
into:
$$(I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}) * \text{vec}(H_{3n}) = \text{vec}(R_{3n}) - \text{vec}(N_y^3)$$
Let
$$\Lambda_3 = I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}$$
<font color='red'>
Above we are assuming the $vec$ operation is taking the column vectors of the matrix. However, it is more easily for us to interprete the results if we assume that $vec$ is taking the row vectors of the matrix. Accodingly $\Lambda_3$ would be changing to
    $$ \Lambda_3 = J_c \otimes I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} + I_{n_c} \otimes C_3^T$$
</font>

In [20]:
Lambda3 = np.kron(J_c,np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c),C3.T)
np.linalg.matrix_rank(Lambda3)

80

Now Since we know that the column vector $-vec(N_y^3)$ is in the column space of matrix $\Lambda_3$ but vector $vec(R_3^n)$ is not. This means vector $vec(R_3^n)$ is in the complement space of the column space of matrix $\Lambda_3$. Since we have already know what the normal form with 3rd order resonance terms looks like (shown below), so we could easiliy know the basis of the space where vector $vec(R_3^n)$ is in.

$$ \dot{z}=i\omega_1z+a_1z^2\bar{z}+a_2zw\bar{w}+\ldots $$ 
$$ \dot{w}=i\omega_2w+b_1w^2\bar{w}+b_2z\bar{z}w+\ldots $$


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

In [21]:
L3 = np.zeros((80,8))
L3[[0,4,21,30],[0,0,0,0]] = 1 # a1_R
L3[[7,9,33,35],[2,2,2,2]] = 1 # a2_R
L3[[56,58,77,79],[4,4,4,4]] = 1 # b1_R
L3[[42,51,63,72],[6,6,6,6]] = 1 # b2_R

L3[[1,10],[1,1]] = -1
L3[[20,24],[1,1]] = 1   #a1_I
L3[[13,15],[3,3]] = -1
L3[[27,29],[3,3]] = 1    #a2_I
L3[[57,59],[5,5]] = -1
L3[[76,78],[5,5]] = 1    #b1_I
L3[[43,52],[7,7]] = -1
L3[[62,71],[7,7]] = 1    #b2_I

# np.linalg.matrix_rank(L3)
print(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))))
assert(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))) == Lambda3.shape[0])

80


In [22]:
# n_c = 4
# for i,v in enumerate([(i,j,k) for i in range(1,n_c+1) for j in range(i,n_c+1) for k in range(j,n_c+1)]):
#     print(i,v)

In [23]:
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
L_inv = np.linalg.inv(L)

U_zero_rows = np.where(abs(np.diag(U)) < 1e-10)[0]
L2 = L_inv[U_zero_rows,:]
theta =np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-1.0841585741707599e+03-2.3709738340847047e-12j],
       [-3.8841899750839139e+03+3.8184873964023091e-12j],
       [-6.7361405904819858e+00+2.2692320808264835e-13j],
       [-1.4713873563043714e+02-1.2656786276665128e-13j],
       [-1.1249526036696388e+01+3.3703632911220552e-14j],
       [ 6.3629878229493819e-01+7.3152214837206380e-14j],
       [ 8.2528140415552809e+02-2.8890093287199104e-12j],
       [ 2.4960747697857223e+03-5.3950422422736655e-13j]])

In [24]:
# array([[-1.0841585741679587e+03],
#        [-3.8841899750754401e+03],
#        [-6.7361405904018978e+00],
#        [-1.4713873563043040e+02],
#        [-1.1249526036663593e+01],
#        [ 6.3629878236300952e-01],
#        [ 8.2528140415351959e+02],
#        [ 2.4960747697811548e+03]])

In [24]:
# pd.DataFrame(U[U_zero_rows,:].T)
np.where(abs(U[U_zero_rows,:]) > 1e-10)

(array([], dtype=int64), array([], dtype=int64))

In [59]:
# fig,ax = plt.subplots(figsize=(10,10))
# ax.spy(U, precision=0.1, markersize=5)
# plt.grid(True)

$$ \dot{r_1}=\mu_1 r_1+a_1^R r_1^3+a_2^R r_1 r_2^2+\ldots $$ 
$$ \dot{r_2}=\mu_2 r_2+b_1^R r_2^3+b_2^R r_1^2 r_2+\ldots $$ 

In [25]:
# a1_R, a2_R, b1_R, b2_R
ab_R = theta[0::2]
ab_R

array([[-1084.15857417076    -2.3709738340847047e-12j],
       [   -6.736140590481986+2.2692320808264835e-13j],
       [  -11.249526036696388+3.3703632911220552e-14j],
       [  825.2814041555281  -2.8890093287199104e-12j]])

In [26]:
ab_R[3]/ab_R[0]

array([-0.7612183529396377])

In [ ]:
# q, r = np.linalg.qr(Lambda3)
# zero_rows = np.where(abs(np.diag(r)) < 1e-10)[0]
# q_inv = q.T
# q2 = q_inv[zero_rows,:]
# theta =np.linalg.inv(q2.dot(L3)).dot(q2).dot(Ny3.reshape((1,-1)).T)
# theta